# 反向传播

---

[反向传播神经网络极简入门](http://www.hankcs.com/ml/back-propagation-neural-network.html)

In [1]:
import math

## 1.Sigmoid

$ f(x) = \frac{ 1 }{1 + e ^ x} $

$ f^`(x) = f(x)(1 - f(x)$

---

In [3]:
def sigmoid(x):
    """
    sigmoid 函数，1/(1+e^-x)
    :param x:
    :return:
    """
    return 1.0/1.0 + math.exp(-x)

def dsigmoid(y):
    """
    sigmoid 函数的导数
    :param y:
    :return:
    """
    return y * (1 - y)

## 2.Tanh

$ f(x) = \frac{1 - e^\left(-x \right)}{1 + e^\left(-x \right)} $

$ f^`(x) = 1 - \left( f(x) \right)^2$

---

In [4]:
def tanh(x):
    """
    sigmoid 函数，tanh 
    :param x:
    :return:
    """
    return math.tanh(x)

def dtanh(y):
    """
    sigmoid 函数的导数
    :param y:
    :return:
    """
    return 1 - y ** 2

## 3.NN


In [6]:
def runNN(self, inputs):
        """
        前向传播进行分类
        :param inputs:输入
        :return:类别
        """
        if len(inputs) != self.ni - 1:
            print('incorrect number of inputs')
 
        for i in range(self.ni - 1):
            self.ai[i] = inputs[i]
 
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum += ( self.ai[i] * self.wi[i][j] )
            self.ah[j] = sigmoid(sum)
 
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum += ( self.ah[j] * self.wo[j][k] )
            self.ao[k] = sigmoid(sum)
 
        return self.ao

In [7]:
def backPropagate(self, targets, N, M):
        """
        后向传播算法
        :param targets: 实例的类别 
        :param N: 本次学习率
        :param M: 上次学习率
        :return: 最终的误差平方和的一半
        """
        # http://www.youtube.com/watch?v=aVId8KMsdUU&feature=BFa&list=LLldMCkmXl4j9_v0HeKdNcRA
 
        # 计算输出层 deltas
        # dE/dw[j][k] = (t[k] - ao[k]) * s'( SUM( w[j][k]*ah[j] ) ) * ah[j]
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k] - self.ao[k]
            output_deltas[k] = error * dsigmoid(self.ao[k])
 
        # 更新输出层权值
        for j in range(self.nh):
            for k in range(self.no):
                # output_deltas[k] * self.ah[j] 才是 dError/dweight[j][k]
                change = output_deltas[k] * self.ah[j]
                self.wo[j][k] += N * change + M * self.co[j][k]
                self.co[j][k] = change
 
        # 计算隐藏层 deltas
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error += output_deltas[k] * self.wo[j][k]
            hidden_deltas[j] = error * dsigmoid(self.ah[j])
 
        # 更新输入层权值
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j] * self.ai[i]
                # print 'activation',self.ai[i],'synapse',i,j,'change',change
                self.wi[i][j] += N * change + M * self.ci[i][j]
                self.ci[i][j] = change
 
        # 计算误差平方和
        # 1/2 是为了好看，**2 是平方
        error = 0.0
        for k in range(len(targets)):
            error = 0.5 * (targets[k] - self.ao[k]) ** 2
        return error